In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


### 基本資料前處理：

In [2]:
import pandas as pd

# 讀取 test.csv 和 train.csv
test_df = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW1/topic-test/test.csv')
train_df = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW1/topic-test/train.csv')

# 創建 'death' 標籤
train_df['death'] = train_df['Death Year'].apply(lambda x: 1 if pd.notna(x) else 0)
train_df.drop(columns=['Death Year', 'Book of Death', 'Death Chapter'], inplace=True)

train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)



# 虛擬變數轉換 (Allegiances)
train_df = pd.get_dummies(train_df, columns=['Allegiances'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Allegiances'], drop_first=True)

# 保持 train_df 和 test_df 的一致性
missing_cols = set(train_df.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0

test_df = test_df[train_df.columns.drop('death')]

# 讀取 character-predictions.csv
character_predictions_df = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW1/dataSets/character-predictions.csv')

# 選擇需要的欄位
character_predictions_selected = character_predictions_df[['name', 'house', 'isMarried', 'isNoble', 'numDeadRelations', 'boolDeadRelations', 'isPopular', 'popularity']]
character_predictions_selected.rename(columns={'name': 'Name'}, inplace=True)

# 合併角色資料
train_df = pd.merge(train_df, character_predictions_selected, on='Name', how='left')
test_df = pd.merge(test_df, character_predictions_selected, on='Name', how='left')

train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

<ipython-input-2-d44a7eca732e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  character_predictions_selected.rename(columns={'name': 'Name'}, inplace=True)


### 戰役資料的合併：

這裡基於家族（house）來將戰役資料合併到角色資料上，每個家族參與的戰役死亡風險被計算並加入到角色資料中。這樣你可以利用戰爭風險來影響生死的預測。

In [3]:
# 讀取 battles.csv
battles_df = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW1/dataSets/battles.csv')

# 選擇相關欄位（簡單處理）
battle_risk = battles_df[['attacker_1', 'defender_1', 'major_death']]
battle_risk['house_involved'] = battle_risk[['attacker_1', 'defender_1']].apply(lambda x: x[0] if pd.notna(x[0]) else x[1], axis=1)

# 計算每個家族的平均戰役死亡風險
battle_risk_summary = battle_risk.groupby('house_involved')['major_death'].mean().reset_index()
battle_risk_summary.rename(columns={'house_involved': 'house', 'major_death': 'battle_risk'}, inplace=True)

<ipython-input-3-39edc87e9502>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  battle_risk['house_involved'] = battle_risk[['attacker_1', 'defender_1']].apply(lambda x: x[0] if pd.notna(x[0]) else x[1], axis=1)
<ipython-input-3-39edc87e9502>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  battle_risk['house_involved'] = battle_risk[['attacker_1', 'defender_1']].apply(lambda x: x[0] if pd.notna(x[0]) else x[1], axis=1)


In [4]:
# 清理 train_df 和 test_df 中的家族名稱
train_df['house'] = train_df['house'].str.replace("House ", "").str.strip()
test_df['house'] = test_df['house'].str.replace("House ", "").str.strip()

# 清理 battle_risk_summary 中的家族名稱
battle_risk_summary['house'] = battle_risk_summary['house'].str.strip()



In [5]:
# 在合併之前，先檢查並刪除已有的 'battle_risk' 欄位
if 'battle_risk' in train_df.columns:
    train_df.drop(columns=['battle_risk'], inplace=True)

if 'battle_risk' in test_df.columns:
    test_df.drop(columns=['battle_risk'], inplace=True)

# 清理 train_df 和 test_df 中的家族名稱
train_df['house'] = train_df['house'].str.replace("House ", "").str.strip()
test_df['house'] = test_df['house'].str.replace("House ", "").str.strip()

# 清理 battle_risk_summary 中的家族名稱
battle_risk_summary['house'] = battle_risk_summary['house'].str.strip()

# 合併戰役風險到 train_df 和 test_df
train_df = pd.merge(train_df, battle_risk_summary, on='house', how='left')
test_df = pd.merge(test_df, battle_risk_summary, on='house', how='left')

# 填補 NaN
train_df['battle_risk'].fillna(0, inplace=True)
test_df['battle_risk'].fillna(0, inplace=True)

# 檢查合併結果
print(train_df[['house', 'battle_risk']].head())


# 填補 NaN
train_df['battle_risk'].fillna(0, inplace=True)
test_df['battle_risk'].fillna(0, inplace=True)

           house  battle_risk
0            NaN        0.000
1            NaN        0.000
2  Night's Watch        0.000
3  Night's Watch        0.000
4          Stark        0.625


<ipython-input-5-ec95b7f514ed>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['battle_risk'].fillna(0, inplace=True)
<ipython-input-5-ec95b7f514ed>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [6]:
# 填補家族風險空值（家族未參與戰役的情況）
train_df['battle_risk'].fillna(0, inplace=True)
test_df['battle_risk'].fillna(0, inplace=True)

# Min-Max Scaler 處理
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
train_df['popularity'] = scaler.fit_transform(train_df[['popularity']])
test_df['popularity'] = scaler.transform(test_df[['popularity']])

# 確保資料一致性，並完成合併
missing_cols = set(train_df.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0

test_df = test_df[train_df.columns.drop('death')]

print("戰役資料成功合併！")

<ipython-input-6-157373c9b36a>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['battle_risk'].fillna(0, inplace=True)
<ipython-input-6-157373c9b36a>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

戰役資料成功合併！


## 年齡分箱

In [7]:
# 假設 'character-predictions.csv' 有一個名為 'age' 的欄位
character_predictions_df = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW1/dataSets/character-predictions.csv')

train_df = pd.merge(train_df, character_predictions_df[['name', 'age']], left_on='Name', right_on='name', how='left')
test_df = pd.merge(test_df, character_predictions_df[['name', 'age']], left_on='Name', right_on='name', how='left')

# 然後進行年齡分箱

In [8]:
# 設定年齡的分箱區間和標籤
bins = [0, 18, 30, 45, 60, 100]  # 分別表示年齡的區間
labels = ['0-18', '19-30', '31-45', '46-60', '60+']

# 檢查 train_df 是否有 'age' 欄位
if 'age' in train_df.columns:
    # 對 train_df 進行年齡分箱
    train_df['age_group'] = pd.cut(train_df['age'], bins=bins, labels=labels, right=False)
    test_df['age_group'] = pd.cut(test_df['age'], bins=bins, labels=labels, right=False)

    # 將年齡分箱結果轉換為虛擬變數
    train_df = pd.get_dummies(train_df, columns=['age_group'], drop_first=True)
    test_df = pd.get_dummies(test_df, columns=['age_group'], drop_first=True)

    # 確保 test_df 和 train_df 的欄位一致
    missing_cols = set(train_df.columns) - set(test_df.columns)
    for col in missing_cols:
        test_df[col] = 0

    test_df = test_df[train_df.columns.drop('death')]

    print("年齡分箱成功，並已轉換為虛擬變數！")

else:
    print("train_df 沒有 'age' 欄位！")

年齡分箱成功，並已轉換為虛擬變數！


## 建置模型

### Random forest (cross validation)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# 準備資料，去掉無用欄位 'Character', 'Name', 'death'
X = train_df.drop(columns=['Character', 'Name', 'death'])
y = train_df['death']

# 檢查資料是否有缺失值，確保特徵值都是數值型
X = pd.get_dummies(X)  # 如果有類別資料，轉換成one-hot encoding
X.fillna(0, inplace=True)  # 填補任何遺漏值
y.fillna(0, inplace=True)  # 填補目標變數的缺失值

# 初始化 Random Forest 模型
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)

# 使用 5 折交叉驗證 (cv=5)
scores = cross_val_score(rf_clf, X, y, cv=5)

# 打印每一折的準確度
print("Cross-validation scores for each fold:", scores)

# 打印平均準確度
print("Average cross-validation accuracy:", np.mean(scores))


Cross-validation scores for each fold: [0.6884058  0.70289855 0.73722628 0.66423358 0.67883212]
Average cross-validation accuracy: 0.6943192637258013


### RF合理的參數組合

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 設置要調整的參數空間
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],  # 樹的數量
    'max_depth': [None, 10, 20, 30, 40, 50],    # 樹的最大深度
    'min_samples_split': [2, 5, 10],            # 節點分裂所需的最小樣本數
    'min_samples_leaf': [1, 2, 4],              # 葉子節點所需的最小樣本數
    'bootstrap': [True, False]                  # 是否使用有放回的抽樣
}

# 設置隨機森林模型
rf_clf = RandomForestClassifier(random_state=42)

# 設置 RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_clf, param_distributions=param_distributions,
                                   n_iter=50,  # 隨機搜尋50組參數
                                   cv=5,       # 5折交叉驗證
                                   verbose=2,  # 顯示搜尋過程
                                   random_state=42,
                                   n_jobs=-1)  # 使用所有可用CPU核心

# 準備資料
X = train_df.drop(columns=['Character', 'Name', 'death'])
y = train_df['death']
X = pd.get_dummies(X)  # 確保所有類別特徵轉為數值型
X.fillna(0, inplace=True)  # 填補遺漏值

# 進行超參數搜尋
random_search.fit(X, y)

# 打印最佳參數
print("Best parameters found: ", random_search.best_params_)

# 打印最佳結果
print("Best cross-validation score: ", random_search.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': True}
Best cross-validation score:  0.7161853379879404


In [11]:
best_rf_clf = RandomForestClassifier(n_estimators=200, min_samples_split=5, min_samples_leaf=1,
                                     max_depth=None, bootstrap=True, random_state=42)
best_rf_clf.fit(X, y)

RandomForestClassifier(min_samples_split=5, n_estimators=200, random_state=42)

In [12]:
# 準備測試資料，移除不需要的 'Character', 'Name' 欄位
X_test = test_df.drop(columns=['Character', 'Name'])

# 進行 one-hot encoding
X_test = pd.get_dummies(X_test)

# 填補遺漏值
X_test.fillna(0, inplace=True)

# 確保測試資料和訓練資料的特徵保持一致
# 新增訓練資料中有但測試資料中缺失的特徵，並將其值設置為 0
missing_cols = set(X.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0

# 移除測試資料中多餘的特徵，保持特徵順序一致
X_test = X_test[X.columns]

# 進行測試集的預測
test_predictions = best_rf_clf.predict(X_test)

<ipython-input-12-6bc41ea5da52>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[col] = 0
<ipython-input-12-6bc41ea5da52>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[col] = 0
<ipython-input-12-6bc41ea5da52>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[col] = 0
<ipytho

### RF output

In [459]:
# 輸出結果為 CSV，使用 'Character' 而非 'Name'
submission = pd.DataFrame({
    'Character': test_df['Character'],  # 使用 'Character' 編號
    'Death': test_predictions
})
submission.to_csv('1corrected_random_forest_submission.csv', index=False)

print("預測結果已保存，並與 sample_submission.csv 格式一致！")

預測結果已保存，並與 sample_submission.csv 格式一致！


### xgboost (cross validation)

In [15]:
!pip install xgboost # install the xgboost library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 5.2 MB/s eta 0:00:00


In [16]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# 設定要調整的參數空間
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],          # 樹的數量
    'max_depth': [3, 4, 5, 6, 7],                       # 樹的最大深度
    'learning_rate': [0.01, 0.1, 0.2, 0.3],             # 學習率
    'min_child_weight': [1, 3, 5],                      # 最小樣本權重
    'subsample': [0.6, 0.8, 1.0],                       # 樣本抽樣比例
    'colsample_bytree': [0.6, 0.8, 1.0]                 # 特徵抽樣比例
}

# 初始化 XGBoost 模型
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 設置 RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_distributions,
                                   n_iter=50,  # 搜尋50組隨機參數
                                   cv=5,       # 5折交叉驗證
                                   verbose=2,  # 顯示搜索過程
                                   random_state=42,
                                   n_jobs=-1)  # 使用所有可用的CPU核心

# 準備訓練資料
X = train_df.drop(columns=['Character', 'Name', 'death'])
y = train_df['death']

# 處理類別變數並填補缺失值
X = pd.get_dummies(X)
X.fillna(0, inplace=True)

# 進行超參數調整
random_search.fit(X, y)

# 打印最佳參數和最佳交叉驗證結果
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:46:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'subsample': 1.0, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.3, 'colsample_bytree': 0.6}
Best cross-validation score:  0.7234422934518142


### xgboost (output)

In [460]:
# 使用最佳參數初始化 XGBoost 模型
best_xgb_clf = xgb.XGBClassifier(subsample=1.0, n_estimators=200, min_child_weight=5,
                                 max_depth=6, learning_rate=0.3, colsample_bytree=0.6,
                                 use_label_encoder=False, eval_metric='logloss', random_state=42)

# 進行模型訓練
best_xgb_clf.fit(X, y)

# 準備測試資料
X_test = test_df.drop(columns=['Character', 'Name'])
X_test = pd.get_dummies(X_test)
X_test.fillna(0, inplace=True)

# 確保 X_test 和 X 的特徵保持一致
missing_cols = list(set(X.columns) - set(X_test.columns))  # 將 set 轉為 list
missing_df = pd.DataFrame(0, index=X_test.index, columns=missing_cols)
X_test = pd.concat([X_test, missing_df], axis=1)

# 保持欄位順序一致
X_test = X_test[X.columns]

# 進行測試集的預測
test_predictions = best_xgb_clf.predict(X_test)

# 輸出結果為 CSV，使用 'Character' 編號
submission = pd.DataFrame({
    'Character': test_df['Character'],  # 使用 'Character' 編號
    'Death': test_predictions
})
submission.to_csv('final_xgboost_submission.csv', index=False)

print("最佳 XGBoost 預測結果已保存！")



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:33:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


最佳 XGBoost 預測結果已保存！


In [17]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# 設定要調整的參數空間，增加複雜度
param_distributions = {
    'n_estimators': [400, 500, 600, 700],          # 增加樹的數量
    'max_depth': [8, 9, 10, 12],                   # 增加樹的最大深度
    'learning_rate': [0.01, 0.05, 0.1],            # 降低學習率
    'min_child_weight': [1, 2, 3],                 # 調整最小樣本權重
    'subsample': [0.8, 0.9, 1.0],                  # 樣本抽樣比例
    'colsample_bytree': [0.7, 0.8, 0.9],           # 特徵抽樣比例
    'gamma': [0, 0.1, 0.2],                        # 控制葉子節點分裂條件
    'reg_alpha': [0, 0.01, 0.1],                   # L1 正則化
    'reg_lambda': [1, 1.5, 2.0]                    # L2 正則化
}

# 初始化 XGBoost 模型
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 設置 RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_distributions,
                                   n_iter=100,  # 搜尋 100 組隨機參數來增加搜索範圍
                                   cv=5,        # 5折交叉驗證
                                   verbose=2,   # 顯示搜索過程
                                   random_state=42,
                                   n_jobs=-1)   # 使用所有可用的CPU核心

# 準備訓練資料
X = train_df.drop(columns=['Character', 'Name', 'death'])
y = train_df['death']

# 處理類別變數並填補缺失值
X = pd.get_dummies(X)
X.fillna(0, inplace=True)

# 進行超參數調整
random_search.fit(X, y)

# 打印最佳參數和最佳交叉驗證結果
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:48:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'subsample': 0.8, 'reg_lambda': 1.5, 'reg_alpha': 0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 12, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.8}
Best cross-validation score:  0.7263620014810114


In [18]:
# 準備測試資料
X_test = test_df.drop(columns=['Character', 'Name'])
X_test = pd.get_dummies(X_test)
X_test.fillna(0, inplace=True)

# 確保測試資料和訓練資料的特徵保持一致
missing_cols = set(X.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0
X_test = X_test[X.columns]

# 使用最佳參數進行最終模型訓練
best_xgb_clf = random_search.best_estimator_

# 進行測試集的預測
test_predictions = best_xgb_clf.predict(X_test)

# 輸出結果為 CSV，使用 'Character' 編號
submission = pd.DataFrame({
    'Character': test_df['Character'],  # 使用 'Character' 編號
    'Death': test_predictions
})
submission.to_csv('xgboost_submission.csv', index=False)

print("預測結果已保存！")

<ipython-input-18-42520c2fad0d>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[col] = 0
<ipython-input-18-42520c2fad0d>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[col] = 0
<ipython-input-18-42520c2fad0d>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[col] = 0
<ipython-i

預測結果已保存！
